In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("CometLanding.csv")

In [3]:
len(df)

77319

In [4]:
df.drop_duplicates(['id_str'], inplace = True)

In [5]:
len(df)

77268

In [6]:
df = df[df['text'].notnull()]

In [7]:
len(df['from_user'].unique())

50195

In [8]:
language = df.groupby('user_lang')

In [9]:
language.size()

user_lang
ar           428
bg             1
ca           309
cs            42
da            89
de          2916
el            29
en         52316
en-AU          1
en-GB         23
en-gb       1972
es          7540
es-MX          2
eu            62
fa             2
fi           108
fil           10
fr          3313
gl            36
he             2
hi             2
hu            41
id            66
it          2664
ja          1514
ko            98
msa            1
nb             1
nl           838
no            36
pl           157
pt           508
pt-PT          1
ro             8
ru           794
sv           126
th            57
tr           761
uk            43
ur             1
vi             1
xx-lc         24
zh-CN          6
zh-Hans        6
zh-cn        285
zh-tw         27
dtype: int64

In [10]:
noRT = df[~df.text.str.startswith('RT', na=False)]

In [11]:
len(noRT)

17269

In [12]:
reply = df[~df.text.str.contains('@', na=False)]

In [13]:
len(reply)

10624